<a href="https://colab.research.google.com/github/KodaiS/Keras-Reproducibility/blob/main/Keras_Reproducibility.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import random

import numpy as np
import tensorflow as tf


# 再現性確保のための設定
# TensorFlow 2.1以降
os.environ['PYTHONHASHSEED'] = '0'  # ハッシュベースの操作を再現可能にする
os.environ['TF_DETERMINISTIC_OPS'] = '1'  # op determinismを有効化

# TensorFlow 2.8.0以降
# tf.config.experimental.enable_op_determinism()  # op determinismを有効化


# データの読み込み
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# 以下のモデルの学習が再現できるか確認
def fit_model(x_train, y_train):
    model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10),
    tf.keras.layers.Softmax()
    ])

    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

    model.compile(optimizer='adam',
                loss=loss_fn,
                metrics=['accuracy'])

    history = model.fit(x_train, y_train, epochs=3, validation_split=0.2)

    return history


# モデルの学習を2回行って比較
# ループ内のモデル作製前にシード値を設定
histories = []

for i in range(2):
    # シード値の設定
    # TensorFlow 2.1以降
    np.random.seed(1)
    random.seed(1)
    tf.random.set_seed(1)
    
    # TensorFlow 2.8.0以降
    # tf.keras.utils.set_random_seed(1)


    print(f'Model {i}')
    histories.append(fit_model(x_train, y_train))


# 結果の確認
print('')
print('Result: {}'.format(histories[0].history == histories[1].history))

Model 0
Epoch 1/3
1500/1500 [==============================] - 7s 4ms/step - loss: 1.5788 - accuracy: 0.9018 - val_loss: 1.5250 - val_accuracy: 0.9431
Epoch 2/3
1500/1500 [==============================] - 6s 4ms/step - loss: 1.5199 - accuracy: 0.9474 - val_loss: 1.5106 - val_accuracy: 0.9555
Epoch 3/3
1500/1500 [==============================] - 6s 4ms/step - loss: 1.5055 - accuracy: 0.9603 - val_loss: 1.5031 - val_accuracy: 0.9615
Model 1
Epoch 1/3
1500/1500 [==============================] - 6s 4ms/step - loss: 1.5788 - accuracy: 0.9018 - val_loss: 1.5250 - val_accuracy: 0.9431
Epoch 2/3
1500/1500 [==============================] - 6s 4ms/step - loss: 1.5199 - accuracy: 0.9474 - val_loss: 1.5106 - val_accuracy: 0.9555
Epoch 3/3
1500/1500 [==============================] - 6s 4ms/step - loss: 1.5055 - accuracy: 0.9603 - val_loss: 1.5031 - val_accuracy: 0.9615

Result: True


In [ ]:
!python -V

Python 3.7.13
